Shantanu Prasad

21070126081

AIML B1

In [ ]:
import numpy as np
import pandas as pd
import nltk
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
df = pd.DataFrame(data)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df.duplicated(subset=['review']).sum()

418

In [ ]:
df.dropna(subset=['review'], inplace=True)

df.drop_duplicates(subset=['review'], inplace=True)

In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df.duplicated(subset=['review']).sum()

0

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

df['review'] = df['review'].apply(tokenize_and_lemmatize)

In [ ]:
import re

def clean_text(text):

    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    text = re.sub(r'<.*?>', '', text)

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^\w\s]', '', text)

    text = ' '.join(text.split())

    return text

df['review'] = df['review'].apply(clean_text)

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))

In [ ]:
print(df['review'])

0        One reviewer ha mentioned watching 1 Oz episod...
1        wonderful little production filming technique ...
2        thought wa wonderful way spend time hot summer...
3        Basically family little boy Jake think zombie ...
4        Petter Mattei Love Time Money visually stunnin...
                               ...                        
49995    thought movie right good job wa nt creative or...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    Catholic taught parochial elementary school nu...
49998    going disagree previous comment side Maltin on...
49999    one expects Star Trek movie high art fan expec...
Name: review, Length: 49582, dtype: object


In [ ]:
df['review'] = df['review'].str.lower()

df['review'].head()

0    one reviewer ha mentioned watching 1 oz episod...
1    wonderful little production filming technique ...
2    thought wa wonderful way spend time hot summer...
3    basically family little boy jake think zombie ...
4    petter mattei love time money visually stunnin...
Name: review, dtype: object

In [ ]:
abbreviation_mapping = {
    'u': 'you',
    'r': 'are',
    'y': 'why',
}

In [ ]:
contraction_mapping = {
    "i'm": "i am",
    "you're": "you are",
    "they're":'they are'
}

In [ ]:
def replace_abbreviations_and_contractions(text):
    for abbrev, replacement in abbreviation_mapping.items():
        text = text.replace(abbrev, replacement)
    for contraction, expanded in contraction_mapping.items():
        text = text.replace(contraction, expanded)
    return text

df['review'] = df['review'].apply(replace_abbreviations_and_contractions)

In [ ]:
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

,review,sentiment
0,one areevieweare ha mentioned watching 1 oz ep...,1
1,wondearefwhyoul little pareodwhyouction filmin...,1
2,thowhyought wa wondearefwhyoul wawhy spend tim...,1
3,basicallwhy familwhy little bowhy jake think z...,0
4,petteare mattei love time monewhy viswhyouallw...,1
...,...,...
49995,thowhyought movie areight good job wa nt caree...,1
49996,bad plot bad dialogwhyoue bad acting idiotic d...,0
49997,catholic tawhyought paareochial elementaarewhy...,0
49998,going disagareee pareeviowhyous comment side m...,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Step 2: Pad sequences to a fixed length
X_train_padded = pad_sequences(X_train_sequences, maxlen= 50, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen= 50, padding='post')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model_1 = Sequential()
model_1.add(Embedding(input_dim= 10000, output_dim= 50, input_length= 50))
model_1.add(LSTM(32))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_1.summary()

model_1.fit(X_train_padded, y_train, batch_size = 32, epochs= 10, validation_data=(X_test_padded, y_test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 50)            500000    
                                                                 
 lstm_3 (LSTM)               (None, 32)                10624     
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1240/1240 [==============================] - 57s 44ms/step - loss: 0.3697 - accuracy: 0.8321 - val_loss: 0.3314 - val_accuracy: 0.8540
Epoch 2/10
1240/1240 [==============================] - 13s 11ms/step - loss: 0.2485 - accuracy: 0.8981 - val_loss: 0.3392 - val_accuracy: 0.8552
Epoch 3/10
1240/1240

In [ ]:
y_pred = model_1.predict(X_test_padded)

310/310 [==============================] - 1s 3ms/step


In [ ]:
tokenizer = Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded_2 = pad_sequences(X_train_sequences, maxlen= 30, padding='post')
X_test_padded_2 = pad_sequences(X_test_sequences, maxlen= 30, padding='post')


In [ ]:
model_2 = Sequential()
model_2.add(Embedding(input_dim= 25000, output_dim= 30, input_length= 30))
model_2.add(LSTM(32, return_sequences=True))
model_2.add(LSTM(32))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_2.summary()

model_2.fit(X_train_padded_2, y_train, batch_size= 32, epochs=10 , validation_data=(X_test_padded_2, y_test))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 30, 30)            750000    
                                                                 
 lstm_4 (LSTM)               (None, 30, 32)            8064      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 766,417
Trainable params: 766,417
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1240/1240 [==============================] - 45s 33ms/step - loss: 0.4184 - accuracy: 0.8028 - val_loss: 0.3768 - val_accuracy: 0.8292
Epoch 2/10
1240/1240 [============

In [ ]:
y_pred_2 = model_1.predict(X_test_padded_2)

310/310 [==============================] - 1s 2ms/step


https://github.com/shantanuprasadbtech/NLP_Assignment2